# Cross-Language Integration
The Wolfram Language has built-in support for various `external evaluators`, as long as they have ZMQ and JSON packages installed.  
We'll look into making python and julia external calls

## Julia Language
First, we check whether or not the Wolfram Language 'knows' of an existing julia binary.

In [ ]:
FindExternalEvaluators["Julia"]

<div class="alert alert-block alert-warning">
<b>Note:</b>
If you're running this on the binder, the above should return the following target: `/srv/julia/bin/julia`. If you're running this locally and the above returns an empty `Dataset`, then we must instruct the Wolfram Language where to look for a binary. We can do this by searching and registering our executable, e.g. on Unix OS by running `which julia` on the command line and then running the following cell:
</div>

In [ ]:
(*
RegisterExternalEvaluator["Julia", "/executable/result/from/which/julia"]
*)

### Single Commands
We can use `ExternalEvaluate` to evaluate commands in the registered evaluator and return the result as a Wolfram Language expression. For example, Julia dictionaries get returned as an `Association`

In [ ]:
ExternalEvaluate["Julia","Dict(\"one\" => 1, \"two\" => 2, \"three\" => 3)"]

<div class="alert alert-block alert-warning">
<b>Note:</b>
If you're running this locally and the above returns an error, you probably need to add the following packages, by running the following on the julia REPL:
</div>

```julia
using Pkg
Pkg.add("ZMQ")
Pkg.add("JSON")
Pkg.add("LinearAlgebra")
```

We can pass in-line template arguments using the following syntax:

In [ ]:
var=1;
ExternalEvaluate["Julia","Dict(\"one\" => <*var*>, \"two\" => 2, \"three\" => 3)"]

We can run some code before running the command, e.g. import some packages:

In [ ]:
ExternalEvaluate[{"Julia", "Prolog" -> "using LinearAlgebra"},
 "A = [1 1 1 1; 2 2 2 2; 3 3 3 3; 4 4 4 4];
  Bidiagonal(A, :U)"]

### Interactive usage
We can spin up a persistent session with `StartExternalSession` and exchange data b/w the two processes:

In [ ]:
juliaSession = StartExternalSession["Julia"]

In [ ]:
juliaAddArguments=ExternalEvaluate[juliaSession,"
function add_arguments(x,y)
  x + y
 end"];
 ExternalEvaluate[juliaSession,"add_arguments(2,2)"]
 juliaAddArguments[3,4]

We can then call julia from within WolframLanguage functions:

In [ ]:
juliaCubicRoot[x_]:= ExternalEvaluate[juliaSession, StringTemplate["cbrt(`1`)"][x]]
juliaCubicRoot[4]

A slightly more convenient form would be to use `ExternalFunction`:

In [ ]:
juliaCbrt=ExternalFunction["Julia", "cbrt"];
juliaCbrt[4]

In [ ]:
Plot[juliaCbrt[x], {x, 0, 5}, MaxRecursion -> 0, PlotPoints -> 50]

We can end a session using `DeleteObject`:

In [ ]:
DeleteObject[juliaSession]

As a last example, let's show how one can activate an environment and write the same discrete dynamical system code we showed earlier.  
First, let's initialize a new persistent julia session and activate the environment located here:

In [ ]:
Environment["REPO_DIR"]

In [ ]:
juliaSession = StartExternalSession[{
"Julia",
"Prolog"->"using Pkg; Pkg.activate(\"/home/jovyan\"); using StaticArrays"
}]

we then define our higher order iteration function:

In [ ]:
dejongJulia=ExternalEvaluate[juliaSession,"

function dejong_eom_fast(u,p)\n
    a,b,c,d = p\n
    @inbounds begin\n
    du1 = sin(a*u[2]) - cos(b*u[1])\n
    du2 = sin(c*u[1]) - cos(d*u[2])\n
    end\n
    SVector{2,Float64}([du1,du2])\n
end\n

function solve_discrete_map_fast(f,u0,p,n)\n
    u = Vector{typeof(u0)}(undef,n)\n
    @inbounds u[1] = u0\n
    @inbounds for i in 1:n-1\n
        u[i+1] = f(u[i],p)\n
    end\n
    u\n
end\n

u0=SVector{2,Float64}([1.0,1.0])\n
function iterate_dejong(p,n)\n
    solve_discrete_map_fast(dejong_eom_fast,u0,p,n)\n
end\n

"]

and finally call it for visualization in the Wolfram Language:

In [ ]:
resultsJL=ExternalEvaluate[juliaSession,"iterate_dejong((-2.0,-2.0,-1.2,2.0),100000)"];
With[{bins=BinCounts[(Flatten/@resultsJL), {-2, 2, 0.005}, {-2, 2, 0.005}]},
  ArrayPlot[Log[bins+1]]]

Note this turns out to be quite slow..  
I suspect this is due to inefficient data transfer. This is partly remedied below in the Python interface.

## Python
Similarly, we can start a python session:

In [ ]:
ExternalEvaluate[{"Python", "Prolog" -> "import numpy as np"}, "np.random.rand(10,3)"]
%//Normal

Note dense data formats like ndarrays get returned as efficient `NumericArray` expressions.

In [ ]:
pySession = StartExternalSession[{"Python", "Prolog" -> "import numpy as np; from numba import njit"}]

In [ ]:
ExternalEvaluate[pySession,"
@njit\n
def dejong_eom(state, args):\n
    a, b, c, d = args\n
    x, y = state\n
    return np.array([np.sin(a*y) - np.cos(b*x),np.sin(c*x) - np.cos(d*y)])\n

@njit\n
def calc_orbit(out, fmap, x0, args):\n
    out[0,:] = x0\n
    for i in range(len(out)-1):\n
        out[i+1,:] = fmap(out[i,:], args)\n"];
        
AbsoluteTiming[
resultsPy=ExternalEvaluate[pySession,"
N = int(10e6)\n
x0 = np.array([-0.3, 0.2])\n
args = (-2.0, -2.0, -1.2, 2.0)\n
out = np.zeros((N, len(x0)))\n

calc_orbit(out, dejong_eom, x0, args)\n
out"];
]

In [ ]:
With[{bins=BinCounts[resultsPy//Normal, {-2, 2, 0.005}, {-2, 2, 0.005}]},
  ArrayPlot[Log[bins+1]]]

### Python's wolframclient library
In python, this interaction can be (almost) two-way using the [wolframclient package](https://reference.wolfram.com/language/WolframClientForPython/).

We start a persistent python session, and start a local persistent WolframLanguange session from within that.

In [ ]:
pySession = StartExternalSession["Python"];
ExternalEvaluate[pySession,"
from wolframclient.evaluation import WolframLanguageSession\n
from wolframclient.language import wl, wlexpr\n
wlSession = WolframLanguageSession()
"]

We can then evaluate WolframLanguage expressions using:

In [ ]:
ExternalEvaluate[pySession,"
wlSession.evaluate(wlexpr('Integrate[Sin[x]^2,x]'))
"]

and access user-defined functions: 

In [ ]:
ExternalEvaluate[pySession,"
from wolframclient.language import Global\n
wlSession.evaluate(wlexpr('function[x_, f_] := f[x]'))
"]

ExternalEvaluate[pySession,"
wlSession.evaluate(Global.function(4.,wl.Sin))
"]

ExternalEvaluate[pySession,"
pyObjectFunction = wlSession.function(wlexpr('function'))\n
pyObjectFunction(4.,wl.Sin)
"]

As a final (useless) example, let's define an `ExternalFunction` mapping to the `pyObjectFunction` we defined above, which then evaluates in then Wolfram Language again:

In [ ]:
functionFromPythonFromWL=ExternalFunction[pySession,"pyObjectFunction"];
functionFromPythonFromWL[4.,Sin]

Unfortunately, this still does not (yet) share live objects back and from the evaluators - but I believe this might be in the works (?)